Suggestions: "https://chatgpt.com/share/68a4c44f-6f88-800a-9d34-c77ada7f5edf"
https://github.com/ONSdigital/address-index-data/blob/develop/DataScience/requirements.txt
https://github.com/J535D165/recordlinkage

In [1]:
import duckdb

# start a DuckDB connection
con = duckdb.connect(database=':memory:')  # or 'mydb.duckdb' if you want persistence


## 2. Register your Parquet directories as tables

DuckDB can treat directories of partitioned Parquet files as a single table.

In [114]:
# Replace with your actual paths

ppd_root = "../data/hf_release/v1/ppd"
epc_root = "../data/hf_release/v1/epc"

con.execute(f"""
    CREATE OR REPLACE VIEW ppd_src AS
    SELECT *
    FROM parquet_scan('{ppd_root}/**/*.parquet', hive_partitioning=1)
""")

con.execute(f"""
    CREATE OR REPLACE VIEW epc_src AS
    SELECT *
    FROM parquet_scan('{epc_root}/**/*.parquet', hive_partitioning=1)
""")

In [ ]:
pp_gl_2025 = con.execute("""
    SELECT
        "Property Type" AS property_type,
        "Postcode"      AS postcode,
        PAON            AS paon,
        SAON            AS saon,
        Street          AS street,
        Locality        AS locality,
        "Town/City"     AS town_city,
        "District"      AS district,
        "County"        AS county
    FROM ppd_src

""").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [107]:
# Quick sanity check: expect only GL / 2025 rows below
pp = con.execute("""
    SELECT 'Postcode', 'Property Type', PAON, SAON, Street, Locality, 'Town/City', 'District', 'County'
    FROM ppd_src
    WHERE sale_year = 2025 AND outcode2 = 'GL' AND Postcode='GL522QZ'

""").fetchdf()
pp[['Property Type', 'PAON', 'SAON', 'Street']]

,Property Type,PAON,SAON,Street
0,F,"THE POND HOUSE, 19",FLAT 3,PITTVILLE CRESCENT
1,F,"JOHN FORBES HOUSE, 15",FLAT 8,PITTVILLE CRESCENT


In [116]:
pp.columns

Index(['Transaction ID', 'Price', 'Date of Transfer', 'Postcode',
       'Property Type', 'Old/New', 'Duration', 'PAON', 'SAON', 'Street',
       'Locality', 'Town/City', 'District', 'County', 'PPD Category Type',
       'Record Status', 'Outward', 'Sector', 'Postcode Sector', 'outcode2',
       'sale_month', 'sale_year'],
      dtype='object')

In [115]:
pp

,Transaction ID,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,Street,...,District,County,PPD Category Type,Record Status,Outward,Sector,Postcode Sector,outcode2,sale_month,sale_year
0,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},470000,2025-01-29,GL522QZ,F,N,L,"THE POND HOUSE, 19",FLAT 3,PITTVILLE CRESCENT,...,CHELTENHAM,GLOUCESTERSHIRE,A,A,GL52,2,GL52 2,GL,01,2025
1,{34222872-4787-4D2B-E063-4704A8C07853},240250,2025-04-04,GL522QZ,F,N,L,"JOHN FORBES HOUSE, 15",FLAT 8,PITTVILLE CRESCENT,...,CHELTENHAM,GLOUCESTERSHIRE,A,A,GL52,2,GL52 2,GL,04,2025


In [108]:
# Quick sanity check: expect only GL / 2025 rows below
ep = con.execute("""
    SELECT *
    FROM epc_src
    WHERE outcode2 = 'GL' AND POSTCODE='GL52 2QZ'

""").fetchdf()
ep[[ 'PROPERTY_TYPE', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3']]

,PROPERTY_TYPE,ADDRESS1,ADDRESS2,ADDRESS3
0,Flat,Flat 4 John Forbes House,15 Pittville Crescent,None
1,House,"19, Pittville Crescent",None,None
2,Flat,Flat 2,"6, Pittville Crescent",None
3,Maisonette,Flat 2,"10, Pittville Crescent",None
4,Flat,Flat 5,John Forbes House,15 Pittville Crescent
...,...,...,...,...
95,Flat,Flat 10,John Forbes House,15 Pittville Crescent
96,House,3 Pittville Crescent,None,None
97,Flat,Flat 4,Scoriton,16 Pittville Crescent
98,Flat,Flat First Floor Front,11 Pittville Crescent,None


In [109]:
ep['col'] = ep['ADDRESS1'] + ' ' + ep['ADDRESS2'] + ' ' + ep['ADDRESS3']

In [113]:
ep.loc[ep['col'].notna(), 'col']

4      Flat 5 John Forbes House 15 Pittville Crescent
5      Flat 7 John Forbes House 15 Pittville Crescent
7      Flat 7 John Forbes House 15 Pittville Crescent
14    Flat 10 John Forbes House 15 Pittville Crescent
19     Flat 3 John Forbes House 15 Pittville Crescent
28              Flat 5 Scoriton 16 Pittville Crescent
29              Flat 1 Scoriton 16 Pittville Crescent
30              Flat 7 Scoriton 16 Pittville Crescent
32         Flat 1 The Pond House 19 Pittville Cresent
35         Flat 2 The Pond House 19 Pittville Cresent
38        Flat 10 The Pond House 19 Pittville Cresent
39         Flat 9 The Pond House 19 Pittville Cresent
40         Flat 6 The Pond House 19 Pittville Cresent
41              Flat 3 Scoriton 16 Pittville Crescent
42         Flat 5 The Pond House 19 Pittville Cresent
43        Flat 4 The Pond House 19  Pittville Cresent
44         Flat 8 The Pond House 19 Pittville Cresent
45         Flat 7 The Pond House 19 Pittville Cresent
46        Flat 11 The Pond H

In [ ]:
"""
PROPERTY_TYPE=Flat -> "F": "Flat/Maisonette",
PAON is house number or name
SOAN is only if there's a sub-building like a Flat

Strategy:
0. If 'Property Type'='F' from ppc_data, then filter epc_data using PROPERTY_TYPE='Flat'
1. Use street from ppd_data and find in all address (ADDRESS1, ADDRESS2, ADDRESS3) but match addresses after removing letters, punctuations etc so addressed only have string. convert street and addresses to upper-case. then filter where-ever string was matched.
2. if flat than use POAN from ppd_data to extract only string, captilise/standardise, and then find in all three addresses with the same pre-processing as in step 1, and filter down further
3.
"""

In [98]:
!pip install splink==3.10.7

ERROR: Could not find a version that satisfies the requirement splink==3.10.7 (from versions: 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.2.1, 0.2.3, 0.2.4, 0.2.5, 0.2.7, 0.2.8, 0.2.9, 0.3.0, 0.3.5, 0.3.7, 0.3.8, 0.3.9, 0.4.0, 0.4.1, 0.4.2, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0rc1, 2.1.0, 2.1.3, 2.1.4, 2.1.5, 2.1.6, 2.1.7, 2.1.8, 2.1.9, 2.1.10, 2.1.11, 2.1.12, 2.1.13, 2.1.14, 3.0.0.dev1, 3.0.0.dev2, 3.0.0.dev3, 3.0.0.dev4, 3.0.0.dev5, 3.0.0.dev6, 3.0.0.dev7, 3.0.0.dev8, 3.0.0.dev9, 3.0.0.dev10, 3.0.0.dev11, 3.0.0.dev12, 3.0.0.dev13, 3.0.0.dev14, 3.0.0.dev15, 3.0.0.dev16, 3.0.0.dev17, 3.0.0.dev18, 3.0.0.dev19, 3.0.0.dev20, 3.0.0.dev21, 3.0.0.dev22, 3.0.0.dev23, 3.0.0.dev24, 3.0.0.dev25, 3.0.0, 3.0.1, 3.1.0, 3.2.0.dev1, 3.2.0, 3.2.1, 3.2.2.dev1, 3.3.0.dev1, 3.3.0.dev2, 3.3.0.dev3, 3.3.0, 3.3.1, 3.3.2, 3.3.3, 3.3.4, 3.3.5, 3.3.6, 3.3.7, 3.3.8, 3.3.9, 3.4.0, 3.4.1, 3.4.2, 3.4.3, 3.4.4, 3.4.5, 3.5.0.dev1, 3.5.0, 3.5.1, 3.5.2, 3.5.3, 3.5.4, 3.6.0, 3.7.0.

In [102]:
import re
import pandas as pd

# ----------------------------
# Normalisation helpers
# ----------------------------

STREET_ABBREVS = {
    "ST": "STREET", "ST.": "STREET",
    "RD": "ROAD", "RD.": "ROAD",
    "AVE": "AVENUE", "AV": "AVENUE", "AV.": "AVENUE",
    "CRES": "CRESCENT", "CRES.": "CRESCENT",
    "CR": "CRESCENT", "CR.": "CRESCENT",
    "LN": "LANE", "LN.": "LANE",
    "PL": "PLACE", "PL.": "PLACE",
    "CT": "COURT", "CT.": "COURT",
    "DR": "DRIVE", "DR.": "DRIVE",
    "GDNS": "GARDENS", "GDNS.": "GARDENS",
    "GR": "GROVE", "GR.": "GROVE"
}

HOUSE_TOKENS = {
    "HOUSE","LODGE","COTTAGE","HALL","COURT","MANSION","MANSIONS","APARTMENTS",
    "APARTMENT","FLATS","BLOCK","TOWER","BUILDING"
}

def up(s):
    if pd.isna(s): return None
    return re.sub(r"\s+", " ", s.upper().strip())

def de_punct(s):
    if s is None: return None
    s = s.replace(",", " ")
    s = re.sub(r"[^\w\s/]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def norm_street(s):
    s = up(de_punct(s))
    if not s: return None
    parts = s.split()
    return " ".join(STREET_ABBREVS.get(w, w) for w in parts)

def extract_first_int(text):
    if not text: return None
    m = re.search(r"\b(\d{1,5})\b", text)
    return int(m.group(1)) if m else None

def extract_flat_number(s):
    if not s: return None
    # FLAT 3, APT 12, FLAT 3/2 -> 3, FLAT FIRST FLOOR FRONT -> None
    m = re.search(r"\b(FLAT|APT|APARTMENT)\s*(\d{1,4})\b", s)
    if m: return int(m.group(2))
    m = re.search(r"\bFLAT\s*(\d{1,4})(?:/\d+)?\b", s)
    return int(m.group(1)) if m else None

def tokens_without_numbers(text):
    if not text: return []
    return [t for t in text.split() if not re.fullmatch(r"\d+", t)]

def looks_like_building_phrase(tokens):
    return any(t in HOUSE_TOKENS for t in tokens)

def paon_to_number_name(paon_raw):
    """
    Input (various): "THE POND HOUSE, 19" or "19" or "19 THE POND HOUSE"
    Output: "19 THE POND HOUSE" or "19"
    """
    s = up(de_punct(paon_raw))
    if not s: return None
    num = extract_first_int(s)
    toks = tokens_without_numbers(s)
    if num is None:
        # No number: leave as just name tokens (rare in PPD but safe)
        return " ".join(toks) if toks else None
    if not toks:
        # Only a number
        return str(num)
    # If tokens contain building keywords, keep them; otherwise keep all non-numeric tokens
    if looks_like_building_phrase(toks):
        name = " ".join(toks)
    else:
        name = " ".join(toks)
    return f"{num} {name}"

def paon_name_only(paon_number_name):
    """
    From "19 THE POND HOUSE" -> "THE POND HOUSE"
    From "19" -> None
    """
    if not paon_number_name: return None
    parts = paon_number_name.split()
    if parts and re.fullmatch(r"\d+", parts[0]):
        name = " ".join(parts[1:]).strip()
        return name or None
    return paon_number_name  # already name-only

def build_building_name_from_any(*parts):
    """
    Join parts, keep only non-numeric tokens, and, if a building keyword is present,
    keep the surrounding phrase; else return all non-numeric tokens (can be None).
    """
    joined = up(de_punct(" ".join(p for p in parts if p)))
    if not joined: return None
    toks = tokens_without_numbers(joined)
    if not toks: return None
    if looks_like_building_phrase(toks):
        return " ".join(toks)
    # No explicit keyword: still return the alpha tokens (helps with names like "SCORITON")
    return " ".join(toks)

def join_nonnull(*vals):
    return " ".join([v for v in vals if v]).strip() or None

# ----------------------------
# PPD preprocessing (uses PAON→NUMBER NAME and NAME-only)
# ----------------------------

def prep_ppd(df):
    df = df.copy()

    # Street
    df["Street_norm"] = df["Street"].map(norm_street)

    # PAON in NUMBER NAME order and components
    df["PAON_raw_norm"] = df["PAON"].map(up).map(de_punct)
    df["PAON_number_name"] = df["PAON_raw_norm"].map(paon_to_number_name)
    df["paon_num"] = df["PAON_number_name"].map(lambda s: int(s.split()[0]) if s and re.fullmatch(r"\d+.*", s) else None)
    df["paon_name"] = df["PAON_number_name"].map(paon_name_only)

    # SAON/flat
    df["SAON_norm"] = df["SAON"].map(up).map(de_punct)
    df["flat_num"]  = df["SAON_norm"].map(extract_flat_number)

    # Building name: prefer PAON name if it looks like a building; else derive from PAON/SAON
    df["building_name"] = df.apply(
        lambda r: r["paon_name"] if r["paon_name"] and looks_like_building_phrase(r["paon_name"].split())
        else build_building_name_from_any(r["PAON_raw_norm"], r["SAON_norm"]),
        axis=1
    )

    # A compact address string for fuzzy backup
    df["full_norm"] = df.apply(
        lambda r: join_nonnull(r["SAON_norm"], r["PAON_number_name"], r["Street_norm"]),
        axis=1
    )

    df["source_dataset"] = "ppd"
    df["unique_id"] = range(1, len(df)+1)
    return df

# ----------------------------
# EPC preprocessing (aligned fields)
# ----------------------------

def prep_epc(df):
    df = df.copy()
    for c in ["ADDRESS1","ADDRESS2","ADDRESS3"]:
        df[f"{c}_norm"] = df[c].map(up).map(de_punct)

    # Street guess
    def street_guess(r):
        for c in ["ADDRESS3_norm","ADDRESS2_norm","ADDRESS1_norm"]:
            s = r[c]
            if s and re.search(r"\b(ROAD|ST|STREET|RD|DRIVE|AV|AVE|AVENUE|LANE|CRES|CRESCENT|COURT|GARDENS|PLACE|GROVE)\b", s):
                return norm_street(s)
        # Fallback to the longest part
        cand = max([r["ADDRESS1_norm"], r["ADDRESS2_norm"], r["ADDRESS3_norm"]], key=lambda x: len(x or ""), default=None)
        return norm_street(cand) if cand else None

    df["Street_norm"] = df.apply(street_guess, axis=1)

    # PAON number & name derived from ADDRESS1/2 (most likely to carry building/number)
    df["alpha_blob"] = df.apply(lambda r: join_nonnull(r["ADDRESS1_norm"], r["ADDRESS2_norm"]), axis=1)
    df["paon_num"] = df["alpha_blob"].map(extract_first_int)

    # Building name from all address lines (name-only, no numbers)
    df["building_name"] = df.apply(
        lambda r: build_building_name_from_any(r["ADDRESS1_norm"], r["ADDRESS2_norm"], r["ADDRESS3_norm"]),
        axis=1
    )

    # Produce EPC's PAON_number_name to match PPD format ("NUMBER NAME")
    def epc_paon_number_name(r):
        if r["paon_num"] and r["building_name"]:
            return f'{r["paon_num"]} {r["building_name"]}'
        elif r["paon_num"]:
            return str(r["paon_num"])
        elif r["building_name"]:
            # Rare: no number present
            return r["building_name"]
        return None

    df["PAON_number_name"] = df.apply(epc_paon_number_name, axis=1)

    # Flat number
    df["flat_num"] = df.apply(
        lambda r: extract_flat_number(join_nonnull(r["ADDRESS1_norm"], r["ADDRESS2_norm"], r["ADDRESS3_norm"])),
        axis=1
    )

    # Fuzzy backup string
    df["full_norm"] = df.apply(
        lambda r: join_nonnull(r["ADDRESS1_norm"], r["ADDRESS2_norm"], r["ADDRESS3_norm"]),
        axis=1
    )

    df["source_dataset"] = "epc"
    df["unique_id"] = range(1_000_001, 1_000_001 + len(df))
    return df

In [103]:
ppd_prepped = prep_ppd(pp)
epc_prepped = prep_epc(ep)

In [104]:
ppd_prepped

,Transaction ID,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,Street,...,PAON_raw_norm,PAON_number_name,paon_num,paon_name,SAON_norm,flat_num,building_name,full_norm,source_dataset,unique_id
0,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},470000,2025-01-29,GL522QZ,F,N,L,"THE POND HOUSE, 19",FLAT 3,PITTVILLE CRESCENT,...,THE POND HOUSE 19,19 THE POND HOUSE,19,THE POND HOUSE,FLAT 3,3,THE POND HOUSE,FLAT 3 19 THE POND HOUSE PITTVILLE CRESCENT,ppd,1
1,{34222872-4787-4D2B-E063-4704A8C07853},240250,2025-04-04,GL522QZ,F,N,L,"JOHN FORBES HOUSE, 15",FLAT 8,PITTVILLE CRESCENT,...,JOHN FORBES HOUSE 15,15 JOHN FORBES HOUSE,15,JOHN FORBES HOUSE,FLAT 8,8,JOHN FORBES HOUSE,FLAT 8 15 JOHN FORBES HOUSE PITTVILLE CRESCENT,ppd,2


In [72]:
for index, row in temp3[['ADDRESS1', 'ADDRESS2', 'ADDRESS3']].iterrows():
    print(row['ADDRESS1'], row['ADDRESS1'], row['ADDRESS1'])

ADDRESS1    Flat 4 John Forbes House
ADDRESS2       15 Pittville Crescent
ADDRESS3                        None
Name: 0, dtype: object
ADDRESS1    19, Pittville Crescent
ADDRESS2                      None
ADDRESS3                      None
Name: 1, dtype: object
ADDRESS1                   Flat 2
ADDRESS2    6, Pittville Crescent
ADDRESS3                     None
Name: 2, dtype: object
ADDRESS1                    Flat 2
ADDRESS2    10, Pittville Crescent
ADDRESS3                      None
Name: 3, dtype: object
ADDRESS1                   Flat 5
ADDRESS2        John Forbes House
ADDRESS3    15 Pittville Crescent
Name: 4, dtype: object
ADDRESS1                   Flat 7
ADDRESS2        John Forbes House
ADDRESS3    15 Pittville Crescent
Name: 5, dtype: object
ADDRESS1       Flat 3 Elgar House
ADDRESS2    2, Pittville Crescent
ADDRESS3                     None
Name: 6, dtype: object
ADDRESS1                   Flat 7
ADDRESS2        John Forbes House
ADDRESS3    15 Pittville Crescent
Name: 7,

In [64]:
temp.columns

Index(['LMK_KEY', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE',
       'BUILDING_REFERENCE_NUMBER', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'POTENTIAL_ENERGY_EFFICIENCY', 'PROPERTY_TYPE', 'BUILT_FORM',
       'INSPECTION_DATE', 'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY',
       'LODGEMENT_DATE', 'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT',
       'ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_CURRENT',
       'ENERGY_CONSUMPTION_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISS_CURR_PER_FLOOR_AREA', 'CO2_EMISSIONS_POTENTIAL',
       'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
       'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL',
       'HOT_WATER_COST_CURRENT', 'HOT_WATER_COST_POTENTIAL',
       'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF', 'MAINS_GAS_FLAG', 'FLOOR_LEVEL',
       'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT', 'MAIN_HEATING_CONTROLS',
       'MULTI_GLAZE_PROPORTION', 'GLAZED_TYPE', 'GLAZED_AREA',
      

In [77]:
!pip install RapidFuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.7 MB/s eta 0:00:00a 0:00:01


In [91]:
# merged = match_merge_by_address(temp, temp3, score_threshold=65)

In [92]:
# merged

,index_df1,Transaction ID,Price,Date of Transfer,Postcode,Property Type,Old/New,Duration,PAON,SAON,...,lodgement_date_1,outcode,addr_key,lodgement_year,outcode2_df2,addr_epc,street_canon_df2,house_no_df2,unit_no_df2,bldg_key_df2
0,0,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},470000,2025-01-29,GL522QZ,F,N,L,"THE POND HOUSE, 19",FLAT 3,...,2025-04-11,GL52,FLAT4THEPONDHOUSE19PITTVILLECRESCENT,2025,GL,Flat 4 The Pond House 19 Pittville Crescent,19 PITTVILLE CRES,4,4,FLAT PITTVILLE POND
1,1,{34222872-4787-4D2B-E063-4704A8C07853},240250,2025-04-04,GL522QZ,F,N,L,"JOHN FORBES HOUSE, 15",FLAT 8,...,2024-08-24,GL52,FLAT10JOHNFORBESHOUSE15PITTVILLECRESCENT,2024,GL,Flat 10 John Forbes House 15 Pittville Crescent,15 PITTVILLE CRES,10,10,FLAT FORBES JOHN PITTVILLE


## 3. Install optional extension for fuzzy matching

In [4]:
# con.execute("INSTALL fuzzystrmatch;")
# con.execute("LOAD fuzzystrmatch;")

## 4. Run queries (examples)

In [5]:
# print(con.execute("DESCRIBE ppd_src").fetchdf())
# print(con.execute("DESCRIBE epc_src").fetchdf())

In [6]:
# sql = """
# WITH
# ppd_n AS (
#   SELECT
#       "Transaction ID" AS transaction_id,
#       "Price" AS price,
#       "Date of Transfer" AS date_of_transfer,
#       UPPER(REPLACE("Postcode", ' ', '')) AS postcode_norm,
#       UPPER(REGEXP_REPLACE(COALESCE("SAON", '') || COALESCE("PAON", '') || COALESCE("Street", ''), '[^A-Z0-9]', '')) AS derived_addr_key,
#       outcode2, sale_month, sale_year
#   FROM ppd_src
# ),
# epc_n AS (
#   SELECT
#       LMK_KEY AS lmk_key,
#       UPPER(REPLACE(POSTCODE, ' ', '')) AS postcode_norm,
#       COALESCE(addr_key,
#                UPPER(REGEXP_REPLACE(COALESCE(ADDRESS1,'') || COALESCE(ADDRESS,'') || COALESCE(ADDRESS2,'') || COALESCE(ADDRESS3,''), '[^A-Z0-9]', ''))
#       ) AS addr_key_std,
#       COALESCE(INSPECTION_DATE, LODGEMENT_DATE) AS epc_date
#   FROM epc_src
# ),
# tier1 AS (
#   SELECT
#       p.transaction_id, e.lmk_key,
#       p.date_of_transfer, e.epc_date
#   FROM ppd_n p
#   JOIN epc_n e
#     ON p.postcode_norm = e.postcode_norm
#    AND p.derived_addr_key = e.addr_key_std
# ),
# tier1_scored AS (
#   SELECT *,
#          ABS(DATE_DIFF('day', epc_date, date_of_transfer)) AS days_diff
#   FROM tier1
#   WHERE (epc_date <= date_of_transfer AND ABS(DATE_DIFF('day', epc_date, date_of_transfer)) <= 365)
#      OR (epc_date >  date_of_transfer AND ABS(DATE_DIFF('day', epc_date, date_of_transfer)) <= 180)
# )
# SELECT *
# FROM tier1_scored
# ORDER BY days_diff
# LIMIT 20;
# """
#
# df = con.execute(sql).fetchdf()
# print(df)


In [53]:
sql = """
WITH
ppd_n AS (
  SELECT
      "Transaction ID" AS transaction_id,
      "Price" AS price,
      "Date of Transfer" AS date_of_transfer,
      "Postcode" AS postcode,
      "PAON", "SAON", "Street", "Locality", "Town/City" AS town_city,
      "District", "County",
      "Property Type" AS property_type,
      "Old/New" AS new_build_flag,
      "Duration" AS tenure,
      outcode2, sale_month, sale_year,
      UPPER(REPLACE("Postcode", ' ', '')) AS postcode_norm,
      UPPER(REGEXP_REPLACE(
          COALESCE("SAON",'') || COALESCE("PAON",'') || COALESCE("Street",''),
          '[^A-Z0-9]', ''
      )) AS derived_addr_key
  FROM ppd_src
  WHERE sale_year = 2025
    AND outcode2 = 'GL'
),
epc_n AS (
  SELECT
      LMK_KEY AS lmk_key,
      POSTCODE,
      ADDRESS1, ADDRESS2, ADDRESS3,
      LOCAL_AUTHORITY, COUNTY, POSTTOWN,
      PROPERTY_TYPE AS property_type_epc,
      BUILT_FORM,
      CURRENT_ENERGY_RATING, POTENTIAL_ENERGY_RATING,
      CURRENT_ENERGY_EFFICIENCY, POTENTIAL_ENERGY_EFFICIENCY,
      TOTAL_FLOOR_AREA,
      CONSTRUCTION_AGE_BAND,
      LODGEMENT_DATE, INSPECTION_DATE,
      UPPER(REPLACE(POSTCODE, ' ', '')) AS postcode_norm,
      COALESCE(addr_key,
        UPPER(REGEXP_REPLACE(
          COALESCE(ADDRESS1,'') || COALESCE(ADDRESS,'') || COALESCE(ADDRESS2,'') || COALESCE(ADDRESS3,''),
          '[^A-Z0-9]', ''
        ))
      ) AS addr_key_std,
      COALESCE(INSPECTION_DATE, LODGEMENT_DATE) AS epc_date
  FROM epc_src
  WHERE outcode2 = 'GL'
),
tier1 AS (
  SELECT
      p.*, e.lmk_key, e.addr_key_std, e.ADDRESS1 AS epc_address1, e.ADDRESS2 AS epc_address2,
      e.ADDRESS3 AS epc_address3, e.POSTCODE AS epc_postcode,
      e.LOCAL_AUTHORITY, e.POSTTOWN, e.COUNTY AS epc_county,
      e.property_type_epc, e.BUILT_FORM,
      e.current_energy_rating, e.potential_energy_rating,
      e.current_energy_efficiency, e.potential_energy_efficiency,
      e.total_floor_area, e.construction_age_band,
      e.lodgement_date, e.inspection_date,
      e.epc_date
  FROM ppd_n p
  FULL JOIN epc_n e
    ON p.postcode_norm = e.postcode_norm
   AND p.derived_addr_key = e.addr_key_std
)
SELECT
    *,
    ABS(DATE_DIFF('day', epc_date, date_of_transfer)) AS days_diff
FROM tier1
WHERE (epc_date <= date_of_transfer AND ABS(DATE_DIFF('day', epc_date, date_of_transfer)) <= 365)
   OR (epc_date >  date_of_transfer AND ABS(DATE_DIFF('day', epc_date, date_of_transfer)) <= 180)
ORDER BY sale_year, sale_month

"""

df = con.execute(sql).fetchdf()

In [54]:
df.shape

(8489, 40)

In [55]:
df

,transaction_id,price,date_of_transfer,postcode,PAON,SAON,Street,Locality,town_city,District,...,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,TOTAL_FLOOR_AREA,CONSTRUCTION_AGE_BAND,LODGEMENT_DATE,INSPECTION_DATE,epc_date,days_diff
0,{2F7F2B42-FBCB-E08F-E063-4804A8C05A49},335000,2025-01-31,GL15PB,61,None,PODSMEAD ROAD,None,GLOUCESTER,GLOUCESTER,...,C,B,70,83,94.0,England and Wales: 1930-1949,2024-09-04,2024-09-04,2024-09-04,149
1,{31C68072-F055-FEE3-E063-4804A8C04F37},264929,2025-01-29,GL46EW,40,None,PENNINGS CLOSE,MATSON,GLOUCESTER,GLOUCESTER,...,B,A,83,97,63.0,2025,2025-01-30,2025-01-30,2025-01-30,1
2,{2F7F2B42-FAD5-E08F-E063-4804A8C05A49},620000,2025-01-24,GL20JF,39,None,CHELTENHAM ROAD,None,GLOUCESTER,GLOUCESTER,...,D,C,65,78,185.0,England and Wales: 1930-1949,2024-04-02,2024-03-19,2024-03-19,311
3,{2F7F2B42-FAE0-E08F-E063-4804A8C05A49},312000,2025-01-24,GL52AR,23,None,LOWER DORRINGTON TERRACE,None,STROUD,STROUD,...,C,B,70,87,89.0,England and Wales: before 1900,2024-07-03,2024-07-03,2024-07-03,205
4,{36A61A94-9139-DEF2-E063-4704A8C046AE},469000,2025-01-24,GL139FT,28,None,NELMES WAY,None,BERKELEY,STROUD,...,B,A,84,93,128.0,2023,2024-11-13,2024-11-13,2024-11-13,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8484,{38EDC0C2-E5B8-0F63-E063-4704A8C00424},106000,2025-06-26,GL13LF,8,None,THE CRESCENT,None,GLOUCESTER,GLOUCESTER,...,D,B,68,83,87.0,England and Wales: before 1900,2024-07-26,2024-07-25,2024-07-25,336
8485,{38EDC0C2-E5C7-0F63-E063-4704A8C00424},138000,2025-06-04,GL207TU,29,None,HAZEL AVENUE,WALTON CARDIFF,TEWKESBURY,TEWKESBURY,...,C,B,79,89,101.0,England and Wales: 2007-2011,2024-06-11,2024-06-11,2024-06-11,358
8486,{38EDC0C2-E4A4-0F63-E063-4704A8C00424},242000,2025-06-04,GL560JH,63,None,CROFT HOLM,None,MORETON-IN-MARSH,COTSWOLD,...,C,B,73,83,105.0,England and Wales: 1996-2002,2025-04-10,2025-04-09,2025-04-09,56
8487,{38EDC0C2-E4C0-0F63-E063-4704A8C00424},255000,2025-06-27,GL114JT,1A,None,UNION STREET,None,DURSLEY,STROUD,...,C,B,75,91,53.0,England and Wales: 1991-1995,2024-08-21,2024-08-21,2024-08-21,310


In [94]:
df[df['postcode'] == 'GL46EW'][['date_of_transfer', 'PAON', 'epc_address2', 'SAON', 'epc_address1', 'Street' ,   'epc_address3']]

,date_of_transfer,PAON,epc_address2,SAON,epc_address1,Street,epc_address3
1,2025-01-29,40,None,None,58 Pennings Close,PENNINGS CLOSE,None
36,2025-01-29,40,None,None,68 Pennings Close,PENNINGS CLOSE,None
102,2025-01-29,40,None,None,64 Pennings Close,PENNINGS CLOSE,None
207,2025-01-29,40,None,None,66 Pennings Close,PENNINGS CLOSE,None
328,2025-01-29,40,None,None,29 Pennings Close,PENNINGS CLOSE,None
...,...,...,...,...,...,...,...
6211,2025-03-21,50,None,None,7 Pennings Cl,PENNINGS CLOSE,None
6215,2025-03-21,50,None,None,12 Pennings Cl,PENNINGS CLOSE,None
6219,2025-03-21,50,None,None,24 Pennings Cl,PENNINGS CLOSE,None
6222,2025-03-21,50,None,None,25 Pennings Close,PENNINGS CLOSE,None


In [32]:
df['derived_addr_key'].head(30)

0      FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT
1                                  16ASHTONCLOSE
2                                   8WALNUTCLOSE
3                                CHERINGTONHOUSE
4     THECOACH HOUSE, 1CANNONS COURTBONDEND ROAD
5     THECOACH HOUSE, 1CANNONS COURTBONDEND ROAD
6                               14WHEATLANDDRIVE
7                        FLAT15ST GEORGES SQUARE
8                        FLAT35ST GEORGES SQUARE
9                               67SNEYDWOOD ROAD
10                        CHANCEHOUSEMAIN STREET
11                        CHANCEHOUSEMAIN STREET
12                      BEECHCROFTSPILLMANSPITCH
13                                   28NELMESWAY
14                        OVERBROOKDRYBROOK ROAD
15                                 4OAKHURSTRISE
16      FLAT17HUCCLECOTE MEWS, 78HUCCLECOTE ROAD
17        48SPRING LAKETHE WATERMARKSTATION ROAD
18                              4HENRIETTASTREET
19                                     8YORKROAD
20                  

In [33]:
df['addr_key_std'].head(30)

0          FLAT10THEPONDHOUSE19PITTVILLECRESCENT
1                         28ASHTONCLOSEABBEYDALE
2                                   6WALNUTCLOSE
3                      STNICHOLASHOUSECHERINGTON
4         TUDORCOTTAGEBONDENDROADUPTONSTLEONARDS
5         TUDORCOTTAGEBONDENDROADUPTONSTLEONARDS
6                               34WHEATLANDDRIVE
7                          FLAT46STGEORGESSQUARE
8                          FLAT46STGEORGESSQUARE
9                                87SNEYDWOODROAD
10              MILLFIELDMAINSTREETCOLNSTALDWYNS
11              MILLFIELDMAINSTREETCOLNSTALDWYNS
12                      BEECHCROFTSPILLMANSPITCH
13                                   26NELMESWAY
14                       3WILLOWDENEDRYBROOKROAD
15                                 9OAKHURSTRISE
16                          FLAT18HUCCLECOTEMEWS
17                       5SPRINGLAKESSTATIONROAD
18                             20HENRIETTASTREET
19                                    8AYORKROAD
20                  

In [39]:
df[df['postcode'] == 'GL522QZ'][['transaction_id', 'lmk_key', 'price', 'postcode_norm', 'derived_addr_key', 'addr_key_std', 'days_diff' ]]

,transaction_id,lmk_key,price,postcode_norm,derived_addr_key,addr_key_std,days_diff
0,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},a3ef0e088658b54dd8225c564810d33f4886c33dfb578f...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",FLAT10THEPONDHOUSE19PITTVILLECRESCENT,175
425,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},c61eb7c264bf835df9d04fc3359f8c4f67c0e67bb37b20...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",3PITTVILLECRESCENT,171
746,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},b778d633275408d3b9cce01b0cb337dc59e847a4907b69...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",FLAT10JOHNFORBESHOUSE15PITTVILLECRESCENT,159
1045,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},87aa0aa7ef680ae0b02155b6b673ad42a4bfcdf66415bc...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",FLAT9THEPONDHOUSE19PITTVILLECRESCENT,362
1823,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},c7ba8212ea1c1e2e264a9a349b43756176caa2b27a5c77...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",FLAT4THEPONDHOUSE19PITTVILLECRESCENT,72
1827,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},90b53ed8df4de713325f4a8819f03ba5fcb142d6bd8789...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",FLATFIRSTFLOORFRONT11PITTVILLECRESCENT,49
1832,{2F7F2B42-FC1B-E08F-E063-4804A8C05A49},856d951dfe2983226f59bd3be5d35238a3349761df8be9...,470000,GL522QZ,"FLAT3THE POND HOUSE, 19PITTVILLE CRESCENT",FLAT4SCORITON16PITTVILLECRESCENT,8
6387,{34222872-4787-4D2B-E063-4704A8C07853},c61eb7c264bf835df9d04fc3359f8c4f67c0e67bb37b20...,240250,GL522QZ,"FLAT8JOHN FORBES HOUSE, 15PITTVILLE CRESCENT",3PITTVILLECRESCENT,236
6501,{34222872-4787-4D2B-E063-4704A8C07853},b778d633275408d3b9cce01b0cb337dc59e847a4907b69...,240250,GL522QZ,"FLAT8JOHN FORBES HOUSE, 15PITTVILLE CRESCENT",FLAT10JOHNFORBESHOUSE15PITTVILLECRESCENT,224
6560,{34222872-4787-4D2B-E063-4704A8C07853},a3ef0e088658b54dd8225c564810d33f4886c33dfb578f...,240250,GL522QZ,"FLAT8JOHN FORBES HOUSE, 15PITTVILLE CRESCENT",FLAT10THEPONDHOUSE19PITTVILLECRESCENT,240


In [24]:
df.shape

(516, 40)